Face mask Tool

✔ Works with MobileNetV2
✔ Uses your two-class dataset
✔ Strong augmentation
✔ EarlyStopping patience
✔ Best model saved
✔ Webcam detects multiple faces + alignment + blur detection

In [6]:
# ------------------------------------------------------
# TRAIN MODEL: MobileNetV2 Face Mask Detection
# ------------------------------------------------------

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Dataset folder structure:
# dataset/with_mask
# dataset/without_mask
data_path = "dataset"

# Image augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.20,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    validation_split=0.2
)

img_size = 224
batch_size = 16

train_gen = datagen.flow_from_directory(
    data_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    data_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Load MobileNetV2
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(img_size, img_size, 3)
)

base_model.trainable = False  # freeze layers

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(2, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# Callbacks
callbacks = [
    EarlyStopping(
        monitor="val_accuracy",
        patience=4,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        "best_mask_mobilenetv2.keras",
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    )
]

# Train
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=12,
    callbacks=callbacks
)

print("Training Complete. Best model saved as best_mask_mobilenetv2.keras")


Found 3077 images belonging to 2 classes.
Found 769 images belonging to 2 classes.


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/12
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step - accuracy: 0.6769 - loss: 0.6189
Epoch 1: val_accuracy improved from None to 0.93238, saving model to best_mask_mobilenetv2.keras
193/193 ━━━━━━━━━━━━━━━━━━━━ 84s 411ms/step - accuracy: 0.7550 - loss: 0.5020 - val_accuracy: 0.9324 - val_loss: 0.2468
Epoch 2/12
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step - accuracy: 0.8809 - loss: 0.3054
Epoch 2: val_accuracy improved from 0.93238 to 0.96099, saving model to best_mask_mobilenetv2.keras
193/193 ━━━━━━━━━━━━━━━━━━━━ 79s 409ms/step - accuracy: 0.8986 - loss: 0.2717 - val_accuracy: 0.9610 - val_loss: 0.1545
Epoch 3/12
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step - accuracy: 0.9200 - loss: 0.2090
Epoch 3: val_accuracy improved from 0.96099 to 0.97789, saving model to best_mask_mobilenetv2.keras
193/193 ━━━━━━━━━━━━━━━━━━━━ 83s 429ms/step - accuracy: 0.9285 - loss: 0.1936 - val_accuracy: 0.9779 - val_loss: 0.1143
Epoch 4/12
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.9507 - loss

WEBCAM MASK DETECTOR

⭐ Features inside:
✔ Multiple faces
✔ Face alignment
✔ Blur detection
✔ Mask classification
✔ Optional placeholder mask-color logic

In [9]:
# ------------------------------------------------------
# FIXED WEBCAM DETECTOR WITH CORRECT LABEL ORDER
# ------------------------------------------------------

import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model("best_mask_mobilenetv2.keras")

# CORRECT CLASS ORDER (alphabetical from flow_from_directory)
labels = ["with_mask", "without_mask"]

img_size = 224

face_detector = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.2, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (img_size, img_size))
        face = face.astype("float32") / 255.0
        face = np.expand_dims(face, axis=0)

        pred = model.predict(face, verbose=0)[0]
        cls = np.argmax(pred)
        label = labels[cls]
        conf = pred[cls]

        # Green for mask, red for no mask
        color = (0,255,0) if label == "with_mask" else (0,0,255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, f"{label} {conf:.2f}",
                    (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Mask Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
